# Discrete time production model with GPU acceleration

## Roman Sigalov

This notebook solves a discrete time production model. It leverages JAX to speed up value function iteration by utilizing GPU by distributing the maximization across potentially multiple GPUs. It is intended to be run on a machine with access to a GPU such as Google Colab or with multiple GPUs such as LambdaLabs servers that allows to use up to 8 GPUs.

This notebook is self contained in that it doesn't import any libraries apart from standard libraries specified below.

**Note**: by default LambdaLabs doesn't have Numba and JAX installed. But it is easy to do within the notebook itself and take very little time (much less than to start-up the instance).

In [ ]:
# Upgrading pip
!pip install --upgrade pip

# Installing numba
!pip install numba

# Installing JAX (following https://github.com/google/jax#pip-installation-gpu-cuda)
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from numba import njit
import timeit
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
import json
import os

# Enabling 64bit floating numbers (otherwise have problem with value
# function iteration but at the same time it makes the code slower)
from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
# Checking GPUs
!nvidia-smi

## Supporting functions

1. `discretizeNormal` -- uses Tauchen method fo discretizing AR(1)
2. `discretizeNormalStVol` -- discretizes a process with state dependent volatility of productivity
3. `computeAPMomentsFromSDF` -- computes asset pricing moments given the transition matrix for the aggregate state and SDF
4. `solveForAgentUtility` -- solves for Epstein-Zin utility and SDF
5. `stationaryDistMarkov` -- calculates stationary distribution of a discrete Markov process given the state transition matrix

In [ ]:
dashed_line_kws = {"color": "black", "linestyle": "--", "alpha": 0.6}

def discretizeNormal(N, mu, rho, sigma, m):
    '''
    Discretization of AR(1) process according following Tauchen

    :param N: number of points
    :param mu: unconditional mean
    :param rho: persistence
    :param sigma: volatility of innovations
    :param m: boundaies in terms of standard deviation, i.e. make a grid for -m*sigma to m*sigma
    :return: a tuple: (grid, transition probabilities matrix, ij index = P(i -> j)
    '''
    Zmax  = m*np.sqrt(sigma*sigma/(1-rho*rho))
    Zmin  = -Zmax
    zstep = 2*Zmax/(N-1)
    Z = np.arange(Zmin,Zmax+zstep,zstep)

    a = (1-rho)*mu
    Z = Z + a/(1-rho)

    # Filling the transition matrix
    Zprob = np.zeros((N, N))
    for j in range(N):
        Zprob[j,0] = norm.cdf((Z[0]-a-rho*Z[j]+zstep/2)/sigma)
        Zprob[j,-1] = 1-norm.cdf((Z[-1]-a-rho*Z[j]-zstep/2)/sigma)
        for k in range(1,N-1):
            Zprob[j,k] = norm.cdf((Z[k]-a-rho*Z[j]+zstep/2)/sigma) - norm.cdf((Z[k]-a-rho*Z[j]-zstep/2)/sigma)

    return Z, Zprob


def discretizeNormalStVol(N, mu, rho, sigma0, b, mmin, mmax):
    '''
    Same as discretizeNormal() but allows for state varying volatili
    to model countercyclical price of risk in a reduced form
    
    For b = 0, volatility = sigma0 = const
    '''
    Zmax  = mmax*np.sqrt(sigma0*sigma0/(1-rho*rho))
    Zmin  = -mmin*np.sqrt(sigma0*sigma0/(1-rho*rho))
    # Zmin  = -Zmax
    zstep = (Zmax-Zmin)/(N-1)
    Z = np.arange(Zmin, Zmax+zstep, zstep)
    
    a = (1-rho)*mu
    Z = Z + a/(1-rho)

    sigmaZ = sigma0*(1+1)/(1 + np.exp(b*Z))

    # For simpler code, calculate bounds of state regions:
    Zlb = np.zeros(N)
    Zub = np.zeros(N)
    Zlb[0], Zub[-1] = -1.0e8, 1.0e8
    Zlb[1:] = Z[1:] - zstep/2
    Zub[:-1] = Z[:-1] + zstep/2

    arg_ub = np.tile(Zub.reshape((1, -1)), (N, 1))
    arg_ub += -(1-rho)*mu - rho*np.tile(Z.reshape((-1, 1)), (1, N))
    arg_ub *= 1/np.tile(sigmaZ.reshape((-1, 1)), (1, N))

    arg_lb = np.tile(Zlb.reshape((1, -1)), (N, 1))
    arg_lb += -(1-rho)*mu - rho*np.tile(Z.reshape((-1, 1)), (1, N))
    arg_lb *= 1/np.tile(sigmaZ.reshape((-1, 1)), (1, N))

    Pz = norm.cdf(arg_ub) - norm.cdf(arg_lb)

    return Z, Pz


def computeAPMomentsFromSDF(Px, M):
    '''
    Using the aggregate state transition matrix Px and SDF matrix M
    to calculate asset pricing moments: risk free rate for each state
    and bound on the maximum achievable Sharpe ratio.
    '''
    # Recomputing moments:
    # Calculating certain moments related to the SDF
    # 1. Risk free rates by state
    Rf = np.array(np.diag(1/Px.dot(M.T) - 1))  # for each state

    # 2. Maximum achievable Sharpe Ratio
    sdM = np.diag(Px.dot(np.power(M.T, 2))) - np.power(np.diag(Px.dot(M.T)),2)
    sdM = np.sqrt(sdM)
    EM = np.diag(Px.dot(M.T))
    maxSR = sdM/EM  # for each state

    return Rf, maxSR


def solveForAgentUtility(xgrid, Px, delta, gamma, psi):
    isUnitPsi = (psi == 1.0)
    Cgrid = np.exp(xgrid)

    if isUnitPsi:
        Uprev = np.power(Cgrid, 1-delta)
    else:
        Uprev = np.power((1-delta)*np.power(Cgrid, 1-1/psi), 1/(1-1/psi))

    dist, tol, it, max_it = 1e6, 1e-8, 0, 10000
    while it < max_it and dist > tol:
        # Can calculate certainity equivalent for U/C separately from consumption growth 
        # since shocks to consumption and consumption growth are uncorrelated
        Uceq = np.power(Px.dot(np.power(Uprev.reshape((-1, 1)), 1-gamma)), 1/(1-gamma))

        # Updating differs depending on whether psi is equal to unity or not
        if isUnitPsi:
            Unew = np.power(Cgrid.reshape((-1, 1)), 1 - delta)*np.power(Uceq, delta)
        else:
            Unew = (1-delta)*np.exp((1-1/psi)*xgrid.reshape((-1, 1)))
            Unew += delta*np.power(Uceq, 1-1/psi)
            Unew = np.power(Unew, 1/(1-1/psi))

        # Calculating sup-distance between next period and current value
        # and updating the next period value
        dist = np.amax(np.abs(Unew - Uprev))
        Uprev = Unew
        it += 1

    U = Uprev.flatten()
    assert it < max_it
    print(f"Agent's problem converged in {it} iterations\n")

    # Constructing an SDF based on consumption and utility
    Mtp1 = np.exp(-(1/psi)*xgrid)*np.power(U, -(gamma-1/psi))
    Mtp1 = Mtp1.reshape((1, -1))

    Uceq = np.power(Px.dot(np.power(U.reshape((-1, 1)), 1-gamma)), 1/(1-gamma))
    Uceq = Uceq.flatten()
    Mt = np.exp(-(1/psi)*xgrid)*np.power(Uceq, -(gamma-1/psi))
    Mt = Mt.reshape((-1, 1))

    M = delta * Mtp1/Mt

    Rf, maxSR = computeAPMomentsFromSDF(Px, M)

    return U, M, Rf, maxSR
    

def stationaryDistMarkov(P):
    '''
    Calculates the vector corresponding to a stationary distribution
    for a discrete state Markov process with transition matrix P where
    P_{ij} = P(i -> j) by calculating the eignevectors and finding the
    one with entries of the same sign (either all positive or all
    negative).

    :param P: transition matrix P_{ij} = P(i -> j)
    :return: vector of stationary distribution
    '''

    w, v = np.linalg.eig(P.T)
    same_sign_index = np.logical_or(
        np.sum(v > 0, axis=0) == P.shape[0],
        np.sum(v < 0, axis=0) == P.shape[0]
    )
    pi = v[:, np.argmax(same_sign_index)]

    # Switching sign if necessary
    if pi[0] < 0:
        pi = -pi

    # Normalizing by the sum
    return pi/np.sum(pi)


# The function compiled with Numba that finds the optimal investment
# at every step of the value function iteration for the equity only model
@njit('float64[:,:,:](float64[:,:,:,:], float64[:,:,:])')
def inner_loop(F, EMV):
    '''Inner loop for value function iteration optimized using numba'''
    Nk, Nx, Ny = EMV.shape
    value = np.zeros(EMV.shape)

    for ix in range(Nx):
        for iy in range(Ny):
            # For given productivity (x, y), k' is an increasing function of k
            # and, hence, we can start searching for optimum k' from prev. max
            # that we store in start_ikp and reuse in the next iteration
            start_ikp = 0
            for ik in range(Nk):
                current_max = -1e10
                for ikp in range(start_ikp, Nk):
                    new_value = F[ik, ikp, ix, iy] + EMV[ikp, ix, iy]
                    if new_value > current_max:
                        current_max = new_value
                        start_ikp = ikp
                    else:
                        break

                value[ik, ix, iy] = current_max

    return value


def bellmanOperatorEquity(V, H, F):
    # Calculating expected continuation value
    EMV = np.tensordot(V, H, axes=([1, 2], [1, 3]))

    # Optimal investment conditional on not exiting
    tomax = F + EMV[None, :, :, :]
    Vmax = np.max(tomax, axis = 1)

    # If Value < 0 => exit
    return np.maximum(Vmax, 0.0)



## `finacingModel` object

The object takes in input parameters and lets to solve the equity and equity+risky debt model and calculate policies. Has the following basic functions

1. `financingModel(params)` -- constructor, create a model and solve for SDF
2. `financingModel.solveForTerminalValue()` -- solve for equity financing only model with no exit
3. `financingModel.calcFlowsEquityIssuance()` -- solve the dividend flows in the abscence of external financing constraints
4. `financingModel.solveEquityIssuance()` -- solve model with costly equity issuance and no debt financing
5. `financingModel.calcPoliciesEquityIssuance()` -- given the solution to costly equity issuance model, calculates policies: investments, dividends and exit decision
6. `financingModel.solveRiskyDebt()` -- solves a model with defaultable debt. **Accelerated with GPUs**
7. `financingModel.calcPoliciesRiskyDebt()` -- calculates policies for the risky debt model

A specific use case is presented below

In [ ]:
class financingModel():
    def __init__(self, params) -> None:

        # Production parameters
        self.delta = params['delta']
        self.cf = params['cf']
        self.theta = params['theta']
        self.phi = params['phi']
        self.phim = params['phim']
        self.phif = params['phif']
        self.alpha = params['alpha']
        self.betax = params['betax']
        self.cd = params['cd']
        self.xid = params['xid']
        self.tauc = params['tauc']
        self.debt_tax_advantage = params["debt_tax_advantage"]
        self.cc = params["cc"] # cost of cash
        self.taue = params['taue']
        self.rcmax = params['rcmax']
        self.cfprop = params['cfprop']

        # Productivity process parameters
        self.Nx, self.Ny = params['Nx'], params['Ny']
        self.mux, self.rhox = params['mux'], params['rhox']
        self.muy, self.rhoy = params['muy'], params['rhoy']
        self.sigmax, self.sigmay = params['sigmax'], params['sigmay']

        # Discretizing using Tauchen method
        self.b = params['b']
        self.xgrid, self.Px = discretizeNormalStVol(self.Nx, self.mux, self.rhox, self.sigmax, self.b, 4, 4)  # Aggregate risk
        self.ygrid, self.Py = discretizeNormal(self.Ny, self.muy, self.rhoy, self.sigmay, 3)  # Idiosyncratic risk

        # Solving for Epstein-Zin agents utility to form the SDF
        self.beta, self.gamma, self.psi = params['beta'], params['gamma'], params['psi']
        self.U, self.M, self.Rf, self.maxSR = solveForAgentUtility(self.xgrid, self.Px, self.beta, self.gamma, self.psi)
        
        # Matrices used for VFI later on
        self.xmat = np.repeat(self.xgrid[:, np.newaxis], self.Ny, axis=1)
        self.ymat = np.repeat(self.ygrid[np.newaxis, :], self.Nx, axis=0)

        # Forming transitional matrix H(x',y'|x,y) = M(x'|x)P(x',y'|x,y)
        self.Hx = np.repeat((self.M*self.Px)[:, :, np.newaxis], self.Ny, axis=2)
        self.Hx = np.repeat(self.Hx[:, :, :, np.newaxis], self.Ny, axis=3)
        self.Hy = np.repeat(self.Py[np.newaxis, :, :], self.Nx, axis=0)
        self.Hy = np.repeat(self.Hy[np.newaxis, :, :, :], self.Nx, axis=0)
        self.H = self.Hx*self.Hy

        self.EM = np.sum(self.H, axis=(1, 3))


    def overrideSDF(self, M):
        # Using this function to, for example, fix a risk free rate
        # constant for every aggregate state. In this case, delta and
        # psi become irrelevant
        
        # Writing SDF
        self.M = M

        # Recomputing moments:
        Rf, maxSR = computeAPMomentsFromSDF(self.Px, M)
        self.Rf = Rf
        self.maxSR = maxSR

        self.Hx = np.repeat((self.M*self.Px)[:, :, np.newaxis], self.Ny, axis=2)
        self.Hx = np.repeat(self.Hx[:, :, :, np.newaxis], self.Ny, axis=3)
        self.Hy = np.repeat(self.Py[np.newaxis, :, :], self.Nx, axis=0)
        self.Hy = np.repeat(self.Hy[np.newaxis, :, :, :], self.Nx, axis=0)
        self.H = self.Hx*self.Hy

        self.EM = np.sum(self.H, axis=(1, 3))


    def solveForTerminalValue(self, max_iter=1000, tolerance=1e-8, kgrid=None, Vprev=None):
        '''Solves for infinite horizon terminal value with an inner loop optimized with numba'''

        # Getting parameters
        delta, cf, theta = self.delta, self.cf, self.theta
        alpha, beta, phi = self.alpha, self.beta, self.phi
        betax, tauc = self.betax, self.tauc
        cfprop = self.cfprop

        if kgrid is None:
            kgrid = np.arange(0.01, 3.0, 0.01)

        Nk = kgrid.shape[0]

        if Vprev is None:
            # Starting conditions (need to expand so it is the same across shocks)
            Vprev = 1/(1 - beta) * (1 - tauc)*(np.power(kgrid, alpha) - cf - delta*kgrid - 0.5*phi*delta*delta*kgrid)
            Vprev = np.repeat(Vprev[:, np.newaxis], self.Nx, axis=1)
            Vprev = np.repeat(Vprev[:, :, np.newaxis], self.Ny, axis=2)

        # Calculating F by utilizing broadcasting (performs much much faster)
        I = kgrid[None, :] - ((1-delta)*kgrid)[:, None]
        InvCost = I + 0.5*phi*np.power(I, 2)/kgrid[:, None]

        F = np.exp(betax*self.xgrid[:, None] + self.ygrid[None, :]) # productivity
        F = F[None, :, :]*np.power(kgrid, alpha)[:, None, None] # times scale
        F = F[:, None, :, :] - InvCost[:, :, None, None] # subtracting investment costs
        F += -cf - cfprop*kgrid[:, None, None, None] # subtracting fixed costs
        F *= (1 - tauc) # subtracting taxes

        # Doing an iteration step
        dist, it = 1e6, 0
        print('Iterating...')
        start = timeit.default_timer()
        while it < max_iter and dist > tolerance:
            if it % 50 == 0:
                print(f'iter={it}, dist={dist}')

            # Calculating continuation value with a tensor product over (x', y') dimensions
            contValue = np.tensordot(Vprev, self.H, axes=([1, 2], [1, 3]))

            # Finding the maximum and updating value
            Vnew = inner_loop(F, contValue)
            dist = np.amax(np.abs(Vnew - Vprev))
            Vprev = Vnew

            it += 1

        stop = timeit.default_timer()
        execution_time = stop - start
        print(f"Firm's problem converged in {execution_time} seconds, {it} iterations\n")

        # Final value function on a grid
        self.TV = Vprev
        self.kgrid = kgrid


    def calcTVDerivatives(self):
        '''
        Discretizes 1st derivative using centered differences of the terminal value
        '''

        # Numerically approximate the 1st derivative of the value function 
        #  * Using forward difference for the lowest capital
        #  * backward for highest
        #  * centered for others
        self.dTVdk = np.zeros((self.TV.shape[0], self.TV.shape[1], self.TV.shape[2]))
        dk = self.kgrid[1] - self.kgrid[0] # Assumes that the grid is uniform
        for i in range(self.Nx):
            for j in range(self.Ny):
                self.dTVdk[0, i, j] = (self.TV[1,i,j] - self.TV[0,i,j])/dk
                self.dTVdk[-1, i, j] = (self.TV[-1,i,j] - self.TV[-2,i,j])/dk
                self.dTVdk[1:-1, i, j] = (self.TV[2:,i,j] - self.TV[0:-2,i,j])/(2*dk)

        self.kgrid1 = self.kgrid  # capital grid for 2nd derivative interpolation

        # Precalculate expectations of the first derivative of the value function
        # for each state (i, j) at t=2 (to avoid calculating an average at each
        # step of the optimization, only interpolation over k will be left).
        self.EdTVdk = np.tensordot(self.dTVdk, self.H, axes=([1, 2], [1, 3]))


    def calcFlowsEquityIssuance(self):
        # Getting parameters ...
        delta, cf, theta = self.delta, self.cf, self.theta
        alpha, beta, phi, phim, phif = self.alpha, self.beta, self.phi, self.phim, self.phif
        cd, xid = self.cd, self.xid
        taue, tauc = self.taue, self.tauc
        betax = self.betax

        # ... and capital grid
        kgrid = self.kgrid
        Nk = kgrid.shape[0]

        # Calculating F by utilizing broadcasting (performs much much faster
        # than repeating the array into a new dimension and then summing)
        I = kgrid[None, :] - ((1-delta)*kgrid)[:, None]
        phi_effective = np.where(kgrid[None, :] >= kgrid[:, None], phi, phim)
        inv_fixed_cost = np.where(kgrid[None, :] == kgrid[:, None], 0.0, phif*kgrid[:, None])
        inv_costs = inv_fixed_cost + I + 0.5*phi_effective*np.power(I/kgrid[:, None], 2)*kgrid[:, None]

        # Calculating dividends
        d = np.exp(betax*self.xgrid[:, None] + self.ygrid[None, :])
        d = d[None, :, :]*np.power(kgrid, alpha)[:, None, None]
        d = d[:, None, :, :] - inv_costs[:, :, None, None]
        d = d - cf
        d = (1 - tauc)*d # subtracting taxes

        # If d < 0 calculating equity issuance costs, if d > 0 calculating taxes
        iss_costs = np.where(d < 0, cd*d - xid, 0.0)
        pay_tax = np.where(d > 0, taue*d, 0.0)
        F = d + iss_costs - pay_tax

        # Writing into the object
        self.d = d
        self.F = F


    def solveEquityIssuance(self, tolerance=1e-8, maxiter=5000):
        print("\nSolving costly equity issuance problem")
        Vprev = self.TV

        H = jax.device_put(self.H)
        F = jax.device_put(self.F)

        # Function to be accelerated. Don't pass H and F since they are constant
        # during the iteration so we can let JAX to store them as constants
        # while compiling the function. This will improve performance.
        def bellmanOperatorEquity(V):
            # Calculating expected continuation value
            EMV = jnp.tensordot(V, H, axes=([1, 2], [1, 3]))

            # Optimal investment conditional on not exiting
            tomax = F + EMV[None, :, :, :]
            Vmax = jnp.max(tomax, axis = 1)

            # If Value < 0 => exit
            return jnp.maximum(Vmax, 0.0)

        # # Precompiling GPU accelerated version of the function
        bjit = jax.jit(bellmanOperatorEquity)
        bjit(Vprev)

        dist, it = 1e6, 0
        print('Iterating...')
        start = timeit.default_timer()
        while it < maxiter and dist > tolerance:
            if it % 50 == 0:
                print(f'iter={it}, dist={dist}')

            Vnew = bjit(Vprev)
            dist = np.max(np.abs(Vnew - Vprev))
            Vprev = Vnew
            it += 1

        stop = timeit.default_timer()
        execution_time = stop - start
        print(f"Firm's problem converged in {execution_time} seconds, {it} iterations, dist = {dist}\n")

        self.Vequity = Vprev


    def calcPoliciesEquityIssuance(self):
        V = self.Vequity
        H = self.H
        F = self.F
        d = self.d
        kgrid = self.kgrid
        cd, xid = self.cd, self.xid

        EMV = np.tensordot(V, H, axes=([1, 2], [1, 3]))

        # Optimal investment conditional on not exiting
        tomax = F + EMV[None, :, :, :]
        ikp_opt = np.argmax(tomax, axis=1)
        Vmax = np.max(tomax, axis=1)
        exit_opt = Vmax < 0.0

        # Calculating dividends
        Nk, Nx, Ny = V.shape
        d_opt = np.empty_like(V)
        kp_opt = np.empty_like(V)
        for ik in range(Nk):
            for ix in range(Nx):
                for iy in range(Ny):
                    d_opt[ik, ix, iy] = d[ik, ikp_opt[ik, ix, iy], ix, iy]
                    kp_opt[ik, ix, iy] = kgrid[ikp_opt[ik, ix, iy]]

        # Calculating flows to equity (div - iss. costs - payout tax)
        iss_costs = np.where(d_opt < 0, cd*d_opt - xid, 0.0)
        pay_tax = np.where(d_opt > 0, self.taue*d_opt, 0.0)
        F_opt = d_opt + iss_costs - pay_tax

        # Writing into the object
        self.F_opt = F_opt
        self.d_opt = d_opt
        self.kp_opt = kp_opt
        self.ikp_opt = ikp_opt
        self.exit_opt = exit_opt

    
    def solveRiskyDebt(self, bgrid=None, Vprev=None, slices=None, tolerance=1e-8, maxiter=250):
        theta, cd, xid = self.theta, self.cd, self.xid
        tauc, taue, rcmax = self.tauc, self.taue, self.rcmax
        cc = self.cc # Cash holding costs
        delta = self.delta
        Nx = self.xgrid.shape[0]
        Ny = self.ygrid.shape[0]
        debt_tax_advantage = self.debt_tax_advantage

        kgrid = self.kgrid
        if bgrid is None:
            bgrid = kgrid
        
        self.bgrid = bgrid
        

        Nk = kgrid.shape[0]
        Nb = bgrid.shape[0]

        H = self.H

        # Expanding starting value to include borrowings: (k, b, x, y)
        if Vprev is None:
            Vprev = np.maximum(self.Vequity[:, None, :, :] - bgrid[None, :, None, None], 0.0)

        Dprev = (Vprev == 0.0)

        # The following arrays do NOT change during iterations => can transfer to
        # GPU before the iteration loop
        bgrid = jax.device_put(bgrid)
        Rf = jax.device_put(self.Rf)
        d_pre_debt = jax.device_put(self.d)

        # Size of slices to pass to the GPU (larger grids require smaller slices)
        # as the GPU may run out of memory. 
        num_gpus = len(jax.devices())
        k_slice_size = 4 # Let each GPU process 4x4 capital/debt points
        b_slice_size = 4 # Let each GPU process 4x4 capital/debt points
        k_maj_slice_size = num_gpus*k_slice_size
        
        # Making sure all of things that should be integers are integers
        assert num_gpus*k_slice_size == k_maj_slice_size
        assert Nk // k_maj_slice_size == Nk/k_maj_slice_size
        assert Nb // b_slice_size == Nb / b_slice_size
        
        # Moving arrays (that are not going to change) to each GPU
        Rf_gpu = [jax.device_put(Rf, device=gpu) for gpu in jax.devices()]
        d_pre_debt_gpu = [jax.device_put(d_pre_debt, device=gpu) for gpu in jax.devices()]
        bgrid_gpu = [jax.device_put(bgrid, device=gpu) for gpu in jax.devices()]

        ################################################################################
        # Defining function that operates for a given pair of (capital, debt)
        # TODO move it outside of here
        def bellmanSlice(iks, ibs, b_proceeds, EMV, Rf, d_pre_debt_gpu, bgrid):
            # Slicing state space
            d_pre_debt_slice = jax.lax.dynamic_slice(d_pre_debt_gpu, (iks, 0, 0, 0), (k_slice_size, Nk, Nx, Ny)) 
            bgrid_slice = jax.lax.dynamic_slice(bgrid, (ibs, ), (b_slice_size, ))

            # Make sure to do everything via broadcasting to speed up things
            d = d_pre_debt_slice[:, None, :, :, :] - ((1 - tauc)*bgrid_slice)[None, :, None, None, None]
            # d = d[:, :, :, None, :, :] + ((1 - tauc)*jnp.maximum(b_proceeds, 0))[None, None, :, :, :, :] # gives (k, b, k', b', x, y)
            d = d[:, :, :, None, :, :] + ((1 - tauc)*b_proceeds)[None, None, :, :, :, :] # gives (k, b, k', b', x, y)
            d = d + debt_tax_advantage*tauc*(bgrid_slice[:, None]*Rf[None, :])[None, :, None, None, :, None]

            # Calculating issuance costs
            iss_costs = (d < 0)*(cd*d - xid)
            pay_tax = (d > 0)*(taue*d)
            F = d + iss_costs - pay_tax

            # Maximizing flow and comparing to zero to determine default decision
            tomax = F + EMV[None, None, :, :, :, :]
            Vmax = jnp.max(tomax, axis=(2, 3))

            return jnp.maximum(Vmax, 0.0)

        ################################################################################
        # Precompiling the function before the first iteration
        # Calculating proceeds from borrowing for given (k', b', x, y)
        b_proceeds = jnp.tensordot(1 - Dprev, H, axes=([2, 3], [1, 3]))*bgrid[None, :, None, None]
        b_proceeds += jnp.tensordot(Dprev, H, axes=([2, 3], [1, 3]))*np.minimum(theta*(1-delta)*kgrid[:, None], rcmax*bgrid[None, :])[:, :, None, None]

        # Calculating expected continuation value
        EMV = jnp.tensordot(Vprev, H, axes=([2, 3], [1, 3]))

        # Transferring to a GPU for precompilation
        EMV_gpu = [jax.device_put(EMV, device=gpu) for gpu in jax.devices()]
        b_proceeds_gpu = [jax.device_put(b_proceeds, device=gpu) for gpu in jax.devices()]

        # Compiling for each GPU
        bellmanSliceJIT = []
        for gpu in jax.devices():
            bellmanSliceJIT.append(jax.jit(bellmanSlice, device=gpu))
            # bellmanSliceJIT.append(jax.jit(bellmanSlice, device=gpu, static_argnames=["Rf", "d_pre_debt_gpu", "bgrid"]))

        for bellman, Rf, d_pre_debt, bgrid in zip(bellmanSliceJIT, Rf_gpu, d_pre_debt_gpu, bgrid_gpu):
            bellman(0, 0, b_proceeds, EMV, Rf, d_pre_debt, bgrid)
        
        ################################################################################
        # Starting the main loop

        dist = 1e6

        start = timeit.default_timer()
        for it in range(1000):
            if it % 10 == 0:
                now = timeit.default_timer()
                print(f"Iteration {it}, dist = {dist}, elapsed = {now-start}")

            # Calculating proceeds from borrowing for given (k', b', x, y)
            b_proceeds = jnp.tensordot(1 - Dprev, H, axes=([2, 3], [1, 3]))*bgrid[None, :, None, None]
            b_proceeds += jnp.tensordot(Dprev, H, axes=([2, 3], [1, 3]))*np.minimum(theta*(1-delta)*kgrid[:, None], rcmax*bgrid[None, :])[:, :, None, None]

            # If saving (b < 0), then earn risk-free rate less the tax holding cost cc
            b_proceeds = np.where(
                bgrid[None, :, None, None] >= 0, 
                b_proceeds, 
                bgrid[None, :, None, None]/(1+Rf)[None, None, :, None]/(1-cc))

            # Calculating expected continuation value
            EMV = jnp.tensordot(Vprev, H, axes=([2, 3], [1, 3]))

            # Transferring arrays that update at every iteration to each GPU
            EMV_gpu = [jax.device_put(EMV, device=gpu) for gpu in jax.devices()]
            b_proceeds_gpu = [jax.device_put(b_proceeds, device=gpu) for gpu in jax.devices()]

            Vnew = np.empty_like(Vprev)

            # Iterating over splits of the state space
            out_gpu = {}
            for ik_maj_group in range(Nk//k_maj_slice_size):
                iks_gpu = []
                ike_gpu = []
                for ibgroup in range(Nb // b_slice_size):
                    ibs, ibe = ibgroup*b_slice_size, (ibgroup+1)*b_slice_size
                    for ikgroup in range(num_gpus):
                        igpu = ikgroup
                        iks, ike = k_maj_slice_size*ik_maj_group + ikgroup*k_slice_size, k_maj_slice_size*ik_maj_group + (ikgroup+1)*k_slice_size
                        out_gpu[(iks, ibs)] = bellmanSliceJIT[igpu](iks, ibs, b_proceeds_gpu[igpu], EMV_gpu[igpu], Rf_gpu[igpu], d_pre_debt_gpu[igpu], bgrid_gpu[igpu])

            # Transferring back to CPU
            Vnew = np.empty((Nk, Nb, Nx, Ny))
            for key in out_gpu.keys():
                iks, ibs = key
                ike, ibe = iks + k_slice_size, ibs + b_slice_size
                Vnew[iks:ike, ibs:ibe, :, :] = np.array(out_gpu[key])
                
            Dnew = (Vnew == 0.0)
            dist = np.max(np.abs(Vnew - Vprev))
            # print(f"   {it}: {dist}")

            Vprev = Vnew
            Dprev = Dnew

            if dist < 1e-8:
                print("Converged!")
                break

        now = timeit.default_timer()
        print(f"{it} iterations in {now - start}, dist = {dist}")

        # Writing into the object
        self.Vdebt = Vnew
        self.Ddebt = Dnew

    def calcPoliciesRiskyDebt(self):
        # Getting dimensions and parameters
        Nk, Nb, Nx, Ny = self.Vdebt.shape
        theta, delta = self.theta, self.delta
        tauc, taue, cd, xid, rcmax = self.tauc, self.taue, self.cd, self.xid, self.rcmax
        cc = self.cc
        debt_tax_advantage = self.debt_tax_advantage
        kgrid, bgrid = self.kgrid, self.bgrid
        Rf = self.Rf

        # Calculating proceeds from borrowing for given (k', b', x, y)
        b_proceeds = jnp.tensordot(1 - self.Ddebt, self.H, axes=([2, 3], [1, 3]))*bgrid[None, :, None, None]
        b_proceeds += jnp.tensordot(self.Ddebt, self.H, axes=([2, 3], [1, 3]))*np.minimum(theta*(1-delta)*kgrid[:, None], rcmax*bgrid[None, :])[:, :, None, None]
        b_proceeds = np.where(
                bgrid[None, :, None, None] >= 0, 
                b_proceeds, 
                bgrid[None, :, None, None]/(1+Rf)[None, None, :, None]/(1-cc))

        # Calculating expected continuation value
        EMV = jnp.tensordot(self.Vdebt, self.H, axes=([2, 3], [1, 3]))

        # Move bgrid, d_pre_debt, b_proceeds, F, EMV to GPU before starting the iteration
        # Transferring matrices to GPU
        b_proceeds = jax.device_put(b_proceeds)
        EMV = jax.device_put(EMV)
        bgrid = jax.device_put(bgrid)
        kgrid = jax.device_put(kgrid)
        Rf = jax.device_put(self.Rf)
        d_pre_debt = jax.device_put(self.d)
        ix_mat = jax.device_put(np.tile(np.array(range(Nx)).reshape((-1, 1)), (1, Ny)))
        iy_mat = jax.device_put(np.tile(np.array(range(Ny)).reshape((1, -1)), (Nx, 1)))

        def calcPoliciesRiskyDebtInner(ik, ib):
            
            # Getting current capital and repayment
            k, b = kgrid[ik], bgrid[ib]

            # Calculating dividend flow including proceeds/payments for debt
            d = d_pre_debt[ik, :, :, :] - (1-tauc)*b
            d = d + debt_tax_advantage*(jnp.maximum(b, 0.0)*tauc*Rf)[None, :, None]
            d = d[:, None, :, :] + (1-tauc)*b_proceeds

            # Calculating issuance costs
            iss_costs = (d < 0)*(cd*d - xid)
            pay_tax = (d > 0)*(taue*d)
            F = d + iss_costs - pay_tax

            # Maximization objective: Flow + Continuation value
            tomax = F + EMV

            # Maximizing flow + continuation value and reshaping indices
            # First, flatten the first two dimensions as argmin only works 
            # with one dimension. Next, unravel 1d index into 2d
            max_ind_flat = jnp.argmax(tomax.reshape((Nk*Nb, Nx, Ny)) , axis=0)
            max_ind_row = max_ind_flat // Nb
            max_ind_col = max_ind_flat - max_ind_row*Nb

            # Write the policies into a single return matrix
            ikp_opt = max_ind_row
            ibp_opt = max_ind_col
            d_opt = d[max_ind_row, max_ind_col, ix_mat, iy_mat]
            exit_opt = tomax[max_ind_row, max_ind_col, ix_mat, iy_mat] <= 0.0
            F_opt = F[max_ind_row, max_ind_col, ix_mat, iy_mat]
            rate_opt = bgrid[max_ind_col]/b_proceeds[max_ind_row, max_ind_col, ix_mat, iy_mat] - 1.0
            Fd_opt = exit_opt*jnp.minimum(theta*(1-delta)*k, rcmax*b)
            Fd_opt = Fd_opt + (1-exit_opt)*(b - b_proceeds[max_ind_row, max_ind_col, ix_mat, iy_mat])

            return ikp_opt, ibp_opt , d_opt, F_opt, Fd_opt, rate_opt, exit_opt

        # Precompiling the function
        inner_jit = jax.jit(calcPoliciesRiskyDebtInner)
        inner_jit(10, 5)

        # Calculating policies for each (ik, ib) separately
        policy_opt = np.zeros((7, Nk, Nb, Nx, Ny))
        for ik in tqdm(range(Nk)):
            for ib in range(Nb):
                ret = inner_jit(ik, ib)
                for ipolicy in range(policy_opt.shape[0]):
                    policy_opt[ipolicy, ik, ib, :, :] = ret[ipolicy]

        # Writing into the object and changing the types
        self.ikp_opt_debt = np.array(policy_opt[0, :, :, :, :], dtype=int)
        self.ibp_opt_debt = np.array(policy_opt[1, :, :, :, :], dtype=int)
        self.d_opt_debt = policy_opt[2, :, :, :, :]
        self.F_opt_debt = policy_opt[3, :, :, :, :]
        self.Fd_opt_debt = policy_opt[4, :, :, :, :]
        self.rate_opt_debt = policy_opt[5, :, :, :, :]
        self.exit_opt_debt = np.array(policy_opt[6, :, :, :, :], dtype=bool)

## Solving the model and saving solution for future use

To allow for future use of models with different parameters, I save the model solution and policies to avoid resolving the model

In [ ]:
def saveModelSolution(model, model_specs, model_params):
    # Getting all shapes
    Nk, Nb, Nx, Ny = model.Vdebt.shape

    # Getting policies for equity-only-model solution
    policy_equity = np.zeros((4, Nk, Nx, Ny))
    policy_equity[0, :, :, :] = model.ikp_opt
    policy_equity[1, :, :, :] = model.d_opt
    policy_equity[2, :, :, :] = model.F_opt
    policy_equity[3, :, :, :] = model.exit_opt

    # Getting policies for risky-debt-model solution
    policy_debt = np.zeros((7, Nk, Nb, Nx, Ny))
    policy_debt[0, :, :, :, :] = model.ikp_opt_debt
    policy_debt[1, :, :, :, :] = model.ibp_opt_debt
    policy_debt[2, :, :, :, :] = model.d_opt_debt
    policy_debt[3, :, :, :, :] = model.F_opt_debt
    policy_debt[4, :, :, :, :] = model.Fd_opt_debt
    policy_debt[5, :, :, :, :] = model.rate_opt_debt
    policy_debt[6, :, :, :, :] = model.exit_opt_debt

    # Saving values and policies
    name = model_specs["name"]
    np.save(f"Vequity_{name}.npy", model.Vequity)
    np.save(f"Vdebt_{name}.npy", model.Vdebt)
    np.save(f"policy_equity_{name}.npy", policy_equity)
    np.save(f"policy_debt_{name}.npy", policy_debt)
    np.save(f"kgrid_{name}.npy", model.kgrid)
    np.save(f"bgrid_{name}.npy", model.bgrid)

    # Saving json files with parameters and description
    to_save_in_json = {}
    to_save_in_json["name"] = model_specs["name"]
    to_save_in_json["desc"] = model_specs["desc"]
    to_save_in_json["params"] = model_params
    json_object = json.dumps(to_save_in_json, indent=4)

    with open(f"model_metadata_{name}.json", "w") as file:
        file.write(json_object)

In [ ]:
baseline_model_params = {
    'mux': -0.1, 'rhox': 0.95, 'sigmax': 0.015, 'b': 0.0, 'Nx': 21, # aggregate state
    'muy': 0.1, 'rhoy': 0.7, 'sigmay': 0.34641016151377546, 'Ny': 13, # idiosyncratic state
    'beta': 0.9645881, 'gamma': 50.0, 'psi': 1.0, # behavioral (SDF) parameters
    'alpha': 0.65, 'cf': 4.0, 'cfprop': 0.0, 'delta': 0.12, 'phi': 2.0, 'phim': 15.0, 'phif':0.0, 'betax': 4.0, # production parameters
    'theta': 0.8, 'cd': 0.2, 'xid': 0.0, # financing parameters
    'tauc': 0.3, 'taue': 0.0, # corporate and payout tax rates
    'rcmax': 0.75, # maximum recuperation
    'debt_tax_advantage': 1.0, # To control whether interest is excluded from taxable profits
    'cc': 0.01 # penalty for holding cash
}

# The following list outlines parameters for different models
model_specs_list = [
    {
        "name": "options_1",
        "desc": "base", 
        "params": {"b": 0.0, "cf": 0.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}},
    {
        "name": "options_2",
        "desc": "FC", 
        "params": {"b": 0.0, "cf": 3.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}},
    {
        "name": "options_3",
        "desc": "CV", 
        "params": {"b": 5.0, "cf": 0.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}},
    {
        "name": "options_4",
        "desc": "EC", 
        "params": {"b": 0.0, "cf": 0.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 0.0}},
    {
        "name": "options_5",
        "desc": "TA", 
        "params": {"b": 0.0, "cf": 0.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_6",
    #     "desc": "FC, CV", 
    #     "params": {"b": 5.0, "cf": 3.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_7",
    #     "desc": "FC, EC", 
    #     "params": {"b": 0.0, "cf": 3.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_8",
    #     "desc": "FC, TA", 
    #     "params": {"b": 0.0, "cf": 3.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_9",
    #     "desc": "CV, EC", 
    #     "params": {"b": 5.0, "cf": 0.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_10",
    #     "desc": "CV, TA", 
    #     "params": {"b": 5.0, "cf": 0.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_11",
    #     "desc": "EC, TA", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_12",
    #     "desc": "FC, CV, EC", 
    #     "params": {"b": 5.0, "cf": 3.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_13",
    #     "desc": "FC, CV, TA", 
    #     "params": {"b": 5.0, "cf": 3.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_14",
    #     "desc": "FC, EC, TA", 
    #     "params": {"b": 0.0, "cf": 3.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_15",
    #     "desc": "CV, EC, TA", 
    #     "params": {"b": 5.0, "cf": 0.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 1.0}} ,
    # {
    #     "name": "options_16",
    #     "desc": "FC, CV, EC, TA", 
    #     "params": {"b": 5.0, "cf": 3.0, "cd": 0.2, "xid": 1.0, "debt_tax_advantage": 1.0}}
    # {
    #     "name": "options_17",
    #     "desc": "low FC", 
    #     "params": {"b": 0.0, "cf": 0.25, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_21",
    #     "desc": "low FC, TA", 
    #     "params": {"b": 0.0, "cf": 0.25, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0}}
    # {
    #     "name": "options_19",
    #     "desc": "prop FC", 
    #     "params": {"b": 0.0, "cf": 0.0, "cfprop": 0.05, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}}
    # {
    #     "name": "options_20",
    #     "desc": "prop FC 2", 
    #     "params": {"b": 0.0, "cf": 0.0, "cfprop": 0.10, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 0.0}}
    # {
    #     "name": "options_22",
    #     "desc": "alt EC 1", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.02, "xid": 0.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_23",
    #     "desc": "alt EC 2", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.05, "xid": 0.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_24",
    #     "desc": "alt EC 3", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.1, "xid": 0.0, "debt_tax_advantage": 0.0}},
    # {
    #     "name": "options_25",
    #     "desc": "alt EC 1, TA", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.02, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_26",
    #     "desc": "alt EC 2, TA", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.05, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_27",
    #     "desc": "alt EC 3, TA", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.1, "xid": 0.0, "debt_tax_advantage": 1.0}},
    # {
    #     "name": "options_28",
    #     "desc": "vanilla", 
    #     "params": {"b": 0.0, "cf": 0.0, "cd": 0.0, "xid": 0.0, "debt_tax_advantage": 1.0, "phi": 5.0, "phim": 5.0}}
]

In [ ]:
bgrid = np.linspace(0.0, 80.0, num=100)
kgrid = np.linspace(0.025, 100.0, num=320)

model_list = []
for i, model_specs in enumerate(model_specs_list):
    print(f" ---- Solving model {i+1} out of {len(model_specs_list)} ----")
    print()
    
    # Getting parameters for the current model
    model_params = baseline_model_params.copy()
    for key in model_specs["params"].keys():
        model_params[key] = model_specs["params"][key]

    model = financingModel(model_params)
    model.solveForTerminalValue(kgrid=kgrid)
    model.calcFlowsEquityIssuance()
    model.solveEquityIssuance()
    model.calcPoliciesEquityIssuance()
    Vstart_debt = np.maximum(model.Vequity[:, None, :, :] - bgrid[None, :, None, None], 0.0)
    model.solveRiskyDebt(bgrid=bgrid, Vprev=Vstart_debt)
    model.calcPoliciesRiskyDebt()
    
    saveModelSolution(model, model_specs, model_params)

    model_list.append(model)

    print()
    print("----"*16)
    print()

---

## An example of solving the model and plotting capital and debt policies

In [ ]:
model_params = {
    'mux': 0.0, 'rhox': 0.95, 'sigmax': 0.015, 'b': 0.0, 'Nx': 21, # aggregate state
    'muy': 0.1, 'rhoy': 0.7, 'sigmay': 0.34641016151377546, 'Ny': 13, # idiosyncratic state
    'beta': 0.9645881, 'gamma': 50.0, 'psi': 1.0, # behavioral (SDF) parameters
    'alpha': 0.65, 'cf': 1.0, 'cfprop': 0.0, 'delta': 0.12, 'phi': 2.0, 'phim': 15.0, 'betax': 2.0, # production parameters
    'theta': 0.8, 'cd': 0.2, 'xid': 1.0, # financing parameters
    'tauc': 0.3, 'taue': 0.12, # corporate and payout tax rates
    'rcmax': 0.75, # maximum recuperation
    'debt_tax_advantage': 1.0, # To control whether interest is excluded from taxable profits
    'cc': 0.0
}

# Baseline aggregate risk parameters
model_params["betax"] = 4.0
model_params["gamma"] = 50.0
model_params["b"] = 0.0
model_params["phi"] = 2.0
model_params["phim"] = 15.0
model_params["phif"] = 0.0

# 1. Solving equity only model
model = financingModel(model_params)
model.solveForTerminalValue(kgrid=np.linspace(0.025, 120.0, num=320))
model.calcTVDerivatives()
model.calcFlowsEquityIssuance()
model.solveEquityIssuance()
model.calcPoliciesEquityIssuance()

# 2. Solving a model with risky debt
bgrid = np.linspace(0.0, 100.0, num=100)
Vstart_debt = np.maximum(model.Vequity[:, None, :, :] - bgrid[None, :, None, None], 0.0)
model.solveRiskyDebt(bgrid=bgrid, Vprev=Vstart_debt)
model.calcPoliciesRiskyDebt()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8), sharey=True)

for ax in axs.flatten():
    ax.axvline(1.0, color="black", linestyle="--", alpha=0.6)    

axs[0, 0].axhline(0.0, color="black", linestyle="--", alpha=0.6)
for iy in [0, 2, 4, 6, 8, 10, 12]:
    lw = 3.0 if iy == 6 else 1.5
    axs[0, 0].plot(model.kgrid, model.ikp_opt[:, 10, iy] - range(model.kgrid.shape[0]), label=f"iy={iy}", linewidth=lw)
axs[0, 0].legend()

axs[0, 1].axhline(0.0, color="black", linestyle="--", alpha=0.6)
for ix in [6, 8, 10, 12, 14]:
    lw = 3.0 if ix == 10 else 1.5
    axs[0, 1].plot(model.kgrid, model.ikp_opt[:, ix, 6] - range(model.kgrid.shape[0]), label=f"ix={ix}", linewidth=lw)
axs[0, 1].legend()

axs[1, 0].axhline(0.0, color="black", linestyle="--", alpha=0.6)
for iy in [0, 2, 4, 6, 8, 10, 12]:
    lw = 3.0 if iy == 6 else 1.5
    axs[1, 0].plot(model.kgrid, model.ikp_opt_debt[:, 0, 10, iy] - range(model.kgrid.shape[0]), label=f"iy={iy}", linewidth=lw)
axs[1, 0].legend()

axs[1, 1].axhline(0.0, color="black", linestyle="--", alpha=0.6)
for ix in [6, 8, 10, 12, 14]:
    lw = 3.0 if ix  == 10 else 1.5
    axs[1, 1].plot(model.kgrid, model.ikp_opt_debt[:, 0, ix, 6] - range(model.kgrid.shape[0]), label=f"ix={ix}", linewidth=lw)
axs[1, 1].legend()

fig.tight_layout()